 ## <center>Exploratory Data Analysis</center>
### <center>Distributions</center>
#### <center> Recommended Source: <a href="https://www.amazon.com/Think-Stats-Exploratory-Data-Analysis/dp/1491907339/ref=sr_1_1?crid=S92SQL92872G&keywords=THINK+STATS&qid=1668390828&sprefix=think+stats%2Caps%2C160&sr=8-1">Link</a></center>

For this guide I will be using my own

While analyzing data one of the best ways of describing a variable is to report values in the dataset and how many times each value appears, in other words view the Distribution of the variable. The most common representation is a <a href="https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/histograms/v/histograms-intro">"histogram"</a>, which is a graph that shows the frequency of each  value.

In [191]:
import pandas as pd
import os
import sys
import ibm_db
from dotenv import load_dotenv
import re
from sqlalchemy import create_engine

In [192]:
def get_watch_data():
    csv_list = os.listdir(os.path.join('datasets','watch_data_dump'))
    watch_data_dict = {}
    for csv in csv_list:

        data = pd.read_csv(os.path.join(os.path.join('datasets','watch_data_dump'), csv), index_col=False, delimiter=',', skiprows=1)

        approved_att_list = [column.replace('com.samsung.shealth.', '').replace('com.samsung.health.', '') for column in data.columns if column not in [
            "heart_rate.deviceuuid",
            "heart_rate.comment",
            "heart_rate.pkg_name",
            "heart_rate.datauuid",
            "custom",
            "deviceuuid",
            "pkg_name",
            "datauuid",
            "extra_data",
            "device_type",
            "source",
            "source_id",
            "source_pkg_name",
            "com.samsung.shealth.calories_burned.pkg_name",
            "com.samsung.shealth.calories_burned.datauuid",
            "com.samsung.shealth.calories_burned.deviceuuid",
            "com.samsung.shealth.calories_burned.deviceuuid",
            "com.samsung.health.sleep.datauuid",
            "com.samsung.health.sleep.pkg_name",
            "com.samsung.health.sleep.deviceuuid",
            "com.samsung.health.sleep.comment",
            "binning_data",
            "tag_id",
            "source_type",
            "com.samsung.health.step_count.datauuid",
            "com.samsung.health.step_count.pkg_name",
            "com.samsung.health.step_count.deviceuuid",
            "com.samsung.health.step_count.sample_position_type",
            "com.samsung.health.step_count.custom",
        ] ]
        data.columns = data.columns.str.replace('com.samsung.shealth.', '', regex=False)
        data.columns = data.columns.str.replace('com.samsung.health.', '', regex=False)
        for column in data.columns:
            if "_time" in column:
                data[column] = pd.to_datetime(data[column])
        for column in data.columns:

            if "_time" in column:
                data = data[(data[column] >= '14/11/2022')]

                break
        data['user_id'] = 1
        data.fillna("", inplace=True)
        result = re.sub(r'[0-9]+', '', csv)
        if 'shealth' in result:
            result = result.replace('shealth.', '')
        elif 'health' in result:
            result = result.replace('health.', '')
        watch_data_dict[result.replace("com.samsung.", "").replace("..csv", "")] = data[approved_att_list]
    return watch_data_dict
watch_data = get_watch_data()


C:\Users\Usuario1\anaconda3\envs\Machine Learning Topics Explanation\lib\site-packages\pandas\core\arraylike.py:60: UserWarning: Parsing '14/11/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return self._cmp_method(other, operator.ge)
C:\Users\Usuario1\anaconda3\envs\Machine Learning Topics Explanation\lib\site-packages\pandas\core\arraylike.py:60: UserWarning: Parsing '14/11/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return self._cmp_method(other, operator.ge)
C:\Users\Usuario1\anaconda3\envs\Machine Learning Topics Explanation\lib\site-packages\pandas\core\arraylike.py:60: UserWarning: Parsing '14/11/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return self._cmp_method(other, operator.ge)
C:\Users\Usuario1\anaconda3\envs\Machine Learning Topics Explanation\lib\site-packages\pandas\core\arraylike.py:60:

In [193]:
watch_data.keys()

dict_keys(['sleep_stage', 'activity.day_summary', 'calories_burned.details', 'exercise', 'exercise.recovery_heart_rate', 'preferences', 'sleep', 'step_daily_trend', 'stress', 'stress.histogram', 'tracker.heart_rate', 'tracker.oxygen_saturation', 'tracker.pedometer_day_summary', 'tracker.pedometer_step_count'])

In [194]:
watch_data['tracker.pedometer_step_count']


,duration,version_code,run_step,walk_step,step_count.start_time,step_count.update_time,step_count.create_time,step_count.count,step_count.speed,step_count.distance,step_count.calorie,step_count.time_offset,step_count.end_time
213,6984,4,0,18,2022-11-14 00:09:00,2022-11-14 00:09:40.901,2022-11-14 00:09:40.901,18,2.138889,14.940000,1.070000,UTC-0600,2022-11-14 00:10:00
219,9553,4,0,15,2022-11-14 01:34:00,2022-11-14 01:34:15.723,2022-11-14 01:34:15.723,15,1.138889,10.880000,1.050000,UTC-0600,2022-11-14 01:35:00
224,10049,4,0,14,2022-11-14 05:35:00,2022-11-14 05:36:31.473,2022-11-14 05:36:31.473,14,0.972222,9.770000,1.020000,UTC-0600,2022-11-14 05:36:00
230,8812,4,0,16,2022-11-14 15:37:00,2022-11-14 15:37:19.648,2022-11-14 15:37:19.504,16,1.362859,12.010254,1.140015,UTC-0600,2022-11-14 15:38:00
235,17642,4,0,29,2022-11-14 23:27:00,2022-11-14 23:27:35.868,2022-11-14 23:27:30.766,29,1.203350,21.230469,2.020081,UTC-0600,2022-11-14 23:28:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,8709,4,0,11,2022-11-16 00:21:00,2022-11-16 00:23:02.160,2022-11-16 00:23:02.160,11,0.861111,7.500000,0.840000,UTC-0600,2022-11-16 00:22:00
455,7292,4,0,12,2022-11-16 01:20:00,2022-11-16 01:22:49.745,2022-11-16 01:22:49.745,12,1.194444,8.710000,0.880000,UTC-0600,2022-11-16 01:21:00
456,10581,4,0,14,2022-11-16 01:28:00,2022-11-16 01:29:52.552,2022-11-16 01:29:52.552,14,0.916667,9.700000,1.190000,UTC-0600,2022-11-16 01:29:00
457,4520,4,0,10,2022-11-16 17:48:00,2022-11-16 17:49:17.153,2022-11-16 17:49:17.153,10,1.750000,7.910000,0.700000,UTC-0600,2022-11-16 17:49:00


In [195]:
def get_scale_data():
    csv_list = os.listdir(os.path.join('datasets','digital_scale_data_dump'))

    data = pd.DataFrame()
    for csv in csv_list:

        data = pd.read_csv(os.path.join(os.path.join('datasets','digital_scale_data_dump'), csv), delimiter=',')
        data['user_id'] = 1

    return data
scale_data = get_scale_data()
scale_data

,Time of Measurement,Weight(lb),BMI,Body Fat(%),Fat-free Body Weight(lb),Subcutaneous Fat(%),Visceral Fat,Body Water(%),Skeletal Muscle(%),Muscle Mass(lb),Bone Mass(lb),Protein(%),BMR(kcal),Metabolic Age,Remarks,user_id
0,2022-11-14 23:34:23,291.2,43.6,26.5,214.0,20.0,24.0,53.1,47.4,203.2,10.8,16.8,2476,39,NaN,1
1,2022-11-15 11:10:38,276.8,41.5,25.3,206.8,19.3,22.0,53.9,48.1,196.6,10.2,17.0,2372,39,NaN,1


### Centralizing Data

I first start by connecting to my ibm db2 database in order to push my data and start working

In [196]:
load_dotenv()
usr = os.getenv("db_usr")
pwd = os.getenv("db_pwd")
hostname = os.getenv("db_hostname")
db_name = os.getenv("db_name")

usr,pwd,hostname,db_name

('root', 'goede', 'localhost', 'health')

In [197]:

connection_uri = f'mysql+pymysql://{usr}:{pwd}@{hostname}/{db_name}'
sqlEngine = create_engine(connection_uri)

conn    = sqlEngine.connect()

In [198]:
forbidden = ['preferences', 'stress_histogram']
for key in watch_data.keys():

    if key.replace('.','_') not in forbidden:
        watch_data[key].to_sql(key.replace('.','_'), con=conn, if_exists='replace', index=False)

stages_data_mapping = {'id':[4001, 4002, 4003, 4004], "stages": ['LIGHT', 'DEEP', 'REM', 'AWAKE']}
sleep_stages = pd.DataFrame(stages_data_mapping)
sleep_stages.to_sql('sleep_stages_mapping', con=conn, if_exists='replace', index=False)

4

In [199]:
scale_data.to_sql('scale_data', con=conn, if_exists='append', index=False)

2